# Reflexion agent

[Reflexion](https://arxiv.org/abs/2303.11366?ref=blog.langchain.dev) by Shinn, et. al., is an architecture designed to learn through verbal feedback and self-reflection. Within reflexion, the actor agent explicitly critiques each response and grounds its criticism in external data. It is forced to generate citations and explicitly enumerate superfluous and missing aspects of the generated response. This makes the content of the reflections more constructive and better steers the generator in responding to the feedback.

In the linked example, we stop after a fixed number of steps, though you can also offload this decision to the reflection LLM call.

https://python.langchain.com/docs/how_to/tool_calling/

## Schemas

In [28]:
from typing import List

from pydantic import BaseModel, Field


class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is  missing")
    superfluous: str = Field(description="Critique of what is superfluous.")
    
    
class AnswerQuestion(BaseModel):
    answer: str = Field(description="~250 word detailed answer to the question")  
    reflection: str = Field(description="Your reflection on the initial answer.")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")  
    
    def _dict(self):
        return {
            'search_queries':self.answer,
            'reflection':self.reflection,
            'search_queries':self.search_queries
        }
        
class ReviserAnswer(BaseModel):
    references: List[str] = Field(description="Citation motivating your update answer")
    

## Tools excutor

In [ ]:
def execute_tools(state: List[BaseMessages]) -> List[ToolMessages]:
    tool_invocation: AIMessages = state[-1]

answer = AnswerQuestion(
        answer="",
        reflection=Reflection(missing="", superfluous=""),
        search_queries=[
            "AI-powered SOC startups funding",
            "AI SOC problem domain specifics",
            "Technologies used by AI-powered SOC startups",
        ],
        id="call_KpYHichFFEmLitHFvFhKy1Ra",
    )



human_message = HumanMessage(
        content="Write about AI-Powered SOC / autonomous soc  problem domain,"
        " list startups that do that and raised capital."
    )


raw_res = execute_tools(
    state=[
        human_message,
        AIMessage(
            content="",
            tool_calls=[
                {
                    "name": AnswerQuestion.__name__,
                    "args": answer.dict(),
                    "id": "call_KpYHichFFEmLitHFvFhKy1Ra",
                }
            ],
        ),
    ]
)
print(raw_res)

 

NameError: name 'BaseMessages' is not defined

## Chains

In [ ]:
import datetime 
from dotenv import load_dotenv 

load_dotenv()

from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser,PydanticToolsParser
from langchain_core.output_parsers import PydanticOutputParser

from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [30]:
llm = ChatOpenAI(model="gpt-4o-mini")
parser = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnswerQuestion])


In [31]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
    "system", 
    """You are expert researcher.
       Current time: {time}
    1. {first_instruction}
    2. Reflect and critique your answer. Be severe to maximizer improvement.
    3. Recommend search queries to resaerch information and improve yours answer.  
    """
        ),
        MessagesPlaceholder(variable_name='messages'),
        ("system", "Answer the user's question above using the required format."),
        
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat()
)

In [32]:
first_response_prompt_template = actor_prompt_template.partial(
    first_instruction = "Provider a detailed ~250 word answer",
)

first_responder =  first_response_prompt_template | llm.bind_tools(
    tools=[AnswerQuestion], tool_choice="AnswerQuestion"
)

In [33]:
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""

revisor = (
    actor_prompt_template.partial(first_instruction=revise_instructions) | 
    llm.bind_tools(tools=[ReviserAnswer], tool_choice='ReviseAnswer')
)


## Main

In [34]:
human_message = HumanMessage(
    content="Write about AI-Powered / autonomus soc problem domain,"
    "list startups that do that and raised capital."
) 

chain = (
    first_response_prompt_template |
    llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion") |
    parser_pydantic   
)

res = chain.invoke(input={"messages":[human_message]})
res

[AnswerQuestion(answer='The problem domain of AI-powered or autonomous Security Operations Centers (SOCs) focuses on enhancing cybersecurity through intelligent automation, reducing the need for human intervention in detecting and responding to threats. Traditional SOCs are often overwhelmed by the volume of security alerts, leading to alert fatigue and potential oversight of critical threats. AI and machine learning technologies can analyze vast amounts of data, identify patterns, and automate responses, thereby improving the efficiency and effectiveness of security operations.\n\nSeveral startups are innovating in this space, leveraging AI to create smarter SOCs. Notable examples include:\n\n1. **Snyk** - A developer-first security platform that helps developers find and fix vulnerabilities in their code. Snyk raised $300 million in a funding round led by Tiger Global in 2021.\n\n2. **Cymulate** - This company provides continuous security validation for organizations, allowing them t

In [36]:
print(res[0].reflection)
print('--------------------------------------\n')
print(res[0].answer)
print('--------------------------------------\n')
print(res[0].search_queries)

print(res[0]._dict())

print(res[0].dict())

While the initial response provides a solid overview of the AI-powered SOC problem domain and lists relevant startups, it lacks depth in explaining how each startup specifically applies AI in their solutions. Additionally, it could benefit from mentioning the challenges faced by traditional SOCs in more detail, as well as potential future trends in the autonomous SOC space. The financial details are informative, but more context about the significance of the funding rounds could enhance the reader's understanding of market dynamics.

Furthermore, the answer could include insights into the competitive landscape and how these startups differentiate themselves from more established players in the cybersecurity field. Including recent statistics or trends in cybersecurity threats would also provide a stronger foundation for the discussion.
--------------------------------------

The problem domain of AI-powered or autonomous Security Operations Centers (SOCs) focuses on enhancing cybersecu